# スクレイピング

### 以下4種類のデータをスクレイピングする。

- レース結果データ	Results.scrape()	results.pickle
- 馬の過去成績データ	HorseResults.scrape()	horse_results.pickle
- 血統データ	Peds.scrape()	peds.pickle
- 払い戻し表データ	Return.scrape()	return_tables.pickle

## モジュールのインポート

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
!pip install optuna
import optuna.integration.lightgbm as lgb_o

## Results.scrape()

メインとなる訓練データである、レース結果データをスクレイピングするメソッド

In [7]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """

        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
          time.sleep(1)
          try:
            url = "https://db.netkeiba.com/race/" + race_id
            #メインとなるテーブルデータを取得
            df = pd.read_html(url)[0]

            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
            #NetKeibaの例     2歳未勝利
            #                 芝右1800m / 天候 : 曇 / 芝 : 良 / 発走 : 09:50
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            info = re.findall(r'\w+', texts)
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text] * len(df)
                if "障" in text:
                    df["race_type"] = ["障害"] * len(df)
                if "m" in text:
                    df["course_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                if text in ["良", "稍重", "重", "不良"]:
                    df["ground_state"] = [text] * len(df)
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    df["weather"] = [text] * len(df)
                if "年" in text:
                    df["date"] = [text] * len(df)

            #馬ID、騎手IDをスクレイピング
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/horse")}
            )
            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])
            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list

            #インデックスをrace_idにする
            df.index = [race_id] * len(df)

            race_results[race_id] = df
          #存在しないrace_idを飛ばす
          except IndexError:
            continue
          #wifiの接続が切れた時などでも途中までのデータを返せるようにする
          except Exception as e:
            print(e)
            break
          #Jupyterで停止ボタンを押した時の対処
          except:
            break

        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])

        return race_results_df

実際に2020年のデータを取得

- 年次
- 場所
  - 01: 札幌 02: 函館 03: 福島 04: 新潟 05: 東京06:中山07: 中京 08: 京都 09:阪神 10: 小倉
- レースの開催回数
- 開催日
- round

In [10]:
race_id_list = []
for place in range(5, 9, 1):
    for kai in range(5, 13, 1):x
        for day in range(1, 13, 1):
            for r in range(5, 13, 1):
                race_id = "2020" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)
		
results = Results.scrape(race_id_list)

## データの保存

In [13]:
results.to_pickle('results.pickle')
results.head(20)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
202005050105,1,6,12,イグナイター,牡2,55,武藤雅,1:38.3,NaN,4.8,2.0,508(0),[西] 牧田和弥,1600,曇,ダート,良,2020年11月7日,2018100916,01169
202005050105,2,5,10,ジンジャーブラッド,牝2,54,丸山元気,1:39.4,7,7.3,4.0,532(0),[東] 林徹,1600,曇,ダート,良,2020年11月7日,2018105089,01127
202005050105,3,8,16,サイレントナイト,牡2,55,岩田望来,1:40.1,4,7.9,5.0,490(0),[東] 矢野英一,1600,曇,ダート,良,2020年11月7日,2018105585,01174
202005050105,4,1,2,サノノヒーロー,牡2,55,津村明秀,1:40.7,3.1/2,27.4,8.0,476(0),[東] 中舘英二,1600,曇,ダート,良,2020年11月7日,2018101033,01092
202005050105,5,2,3,シュルードアイズ,牡2,55,松山弘平,1:41.0,1.3/4,5.1,3.0,486(0),[東] 牧光二,1600,曇,ダート,良,2020年11月7日,2018100859,01126
202005050105,6,1,1,スキャッターシード,牝2,53,斎藤新,1:42.4,9,4.1,1.0,524(0),[東] 斎藤誠,1600,曇,ダート,良,2020年11月7日,2018105420,01178
202005050105,7,7,14,アウスドルック,牝2,54,井上敏樹,1:42.9,3,151,13.0,410(0),[東] 加藤士津,1600,曇,ダート,良,2020年11月7日,2018106167,01151
202005050105,8,7,13,アマラスボーイ,牡2,55,内田博幸,1:43.0,3/4,22.4,7.0,458(0),[東] 松山将樹,1600,曇,ダート,良,2020年11月7日,2018100138,00422
202005050105,9,6,11,ファビュラスナイト,牡2,55,北村宏司,1:43.1,クビ,8.5,6.0,490(0),[東] 高橋裕,1600,曇,ダート,良,2020年11月7日,2018100252,01043
202005050105,10,3,5,ディスコード,牝2,54,嘉藤貴行,1:43.9,5,151.5,14.0,536(0),[東] 高柳瑞樹,1600,曇,ダート,良,2020年11月7日,2018100680,01051


## HorseResults.scrape()

馬の過去成績データをスクレイピングするメソッド



In [11]:
#馬の過去成績データを処理するクラス
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        """
        馬の過去成績データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        horse_results_df : pandas.DataFrame
            全馬の過去成績データをまとめてDataFrame型にしたもの
        """

        #horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = 'https://db.netkeiba.com/horse/' + horse_id
                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0]=='受賞歴':
                    df = pd.read_html(url)[4]
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる        
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])

        return horse_results_df